In [1]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time
import datetime as dt 
import pymongo
from pymongo import MongoClient
import numpy as np
pd.options.mode.chained_assignment = None 

In [28]:
trendings_keywords = pd.read_csv('../Data/trendings.csv')
trendings_keywords['Sector'].value_counts()

Neutro       91
Derecha      42
Izquierda    40
Name: Sector, dtype: int64

In [10]:
trendings_keywords[['trending','Sector']]

,trending,Sector
0,#NosUnimosONosJodemos,Derecha
1,#FedericoEsColombia,Derecha
2,#LoPeorDeEstasElecciones,Neutro
3,#MiVotoEsPetroYFrancia,Izquierda
4,#MiVotoEsPetroYFrancia,Izquierda
...,...,...
168,#DePetroEspero,Neutro
169,#PetroNecesita,Neutro
170,#EsMuyImportante,Neutro
171,#PetroBastaYa,Neutro


In [57]:
df_list = []
for i in range(len(trendings_keywords)):
    start_date = trendings_keywords['date'][i]
    newpath = '../Data/Trendings/'+ start_date + '/'
    trending = trendings_keywords['trending'][i]
    file_name = start_date + '__' +  trending + '.gzip'
    trendings_df = pd.read_parquet(newpath + file_name)
    trendings_df['Trending'] =  trending 
    df_list.append(trendings_df)
full_df = pd.concat(df_list)
full_df = full_df.reset_index(drop=True)
full_df.to_parquet('../Data/Full_dataset/full_data.gzip',
          compression='gzip')

In [2]:
full_df = pd.read_parquet('../Data/Full_dataset/full_data.gzip')

In [3]:
full_df

,lang,author_id,id,created_at,text,conversation_id,in_reply_to_user_id,date,retweet_count,reply_count,like_count,quote_count,place_id,Trending,coordinates.type,coordinates.coordinates
0,es,1143717800,1528164731630018561,2022-05-22 00:03:58+00:00,No le creo al Registrador Vega. \n#NosUnimosON...,1528164731630018561,NaN,2022-05-22T00:00:00Z,27.0,1.0,41.0,0.0,None,#NosUnimosONosJodemos,None,None
1,es,93504770,1528163832690003969,2022-05-22 00:00:24+00:00,#NosUnimosONosJodemos\nLa revista Semana con s...,1528163832690003969,NaN,2022-05-22T00:00:00Z,25.0,2.0,58.0,0.0,None,#NosUnimosONosJodemos,None,None
2,es,1143717800,1528164148445585413,2022-05-22 00:01:39+00:00,@jarizabaletaf Federico Gutierrez Presidente d...,1528149422806384641,965331540.0,2022-05-22T00:00:00Z,17.0,1.0,35.0,0.0,None,#NosUnimosONosJodemos,None,None
3,es,61589689,1528164772281196544,2022-05-22 00:04:08+00:00,#NosUnimosONosJodemos porque no queremos en Co...,1528164772281196544,NaN,2022-05-22T00:00:00Z,20.0,1.0,21.0,0.0,None,#NosUnimosONosJodemos,None,None
4,es,1143717800,1528163966429569025,2022-05-22 00:00:56+00:00,@jarizabaletaf #NosUnimosONosJodemos \nVamos t...,1527970345164185600,965331540.0,2022-05-22T00:00:00Z,9.0,0.0,13.0,0.0,None,#NosUnimosONosJodemos,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584996,es,1275895677239857153,1539759865643601920,2022-06-22 23:58:53+00:00,#LosImpuestosDePetro deberían también ser apli...,1539759865643601920,NaN,2022-06-22T23:50:00Z,19.0,6.0,61.0,2.0,None,#LosImpuestosDePetro,None,None
584997,es,756689409773277184,1539759241808027648,2022-06-22 23:56:25+00:00,TODO SE SUPO - SUCEDE A ESTA HORA - COLOMBIA H...,1539759241808027648,NaN,2022-06-22T23:50:00Z,0.0,0.0,1.0,0.0,None,#LosImpuestosDePetro,None,None
584998,es,87009246,1539758385566031874,2022-06-22 23:53:01+00:00,Los veo muy preocupados por reforma tributaria...,1539758385566031874,NaN,2022-06-22T23:50:00Z,0.0,1.0,0.0,0.0,None,#LosImpuestosDePetro,None,None
584999,es,760914847102070784,1539758097677312004,2022-06-22 23:51:52+00:00,#LosImpuestosDePetro para cuando los congresis...,1539758097677312004,NaN,2022-06-22T23:50:00Z,0.0,0.0,0.0,0.0,None,#LosImpuestosDePetro,None,None


In [4]:
def cleaning(df, prop, words):
    df['Number_words'] = df['text'].str.count(' ') + 1
    df['Number_non_text'] = df['text'].str.count('#|@|http')
    df['non_text_proportion'] = df['Number_non_text']/df['Number_words']
    df = df[(df['non_text_proportion'] <= prop) & (df['Number_words'] > words)]
    df = df.reset_index(drop=True)
    return df

In [5]:
cleaned = cleaning(full_df, 0.2, 5)
#cleaned.to_parquet('../Data/Full_dataset/data_filtered.gzip',
          #compression='gzip')

In [6]:
cleaned

,lang,author_id,id,created_at,text,conversation_id,in_reply_to_user_id,date,retweet_count,reply_count,like_count,quote_count,place_id,Trending,coordinates.type,coordinates.coordinates,Number_words,Number_non_text,non_text_proportion
0,es,1143717800,1528164731630018561,2022-05-22 00:03:58+00:00,No le creo al Registrador Vega. \n#NosUnimosON...,1528164731630018561,NaN,2022-05-22T00:00:00Z,27.0,1.0,41.0,0.0,None,#NosUnimosONosJodemos,None,None,7,1,0.142857
1,es,93504770,1528163832690003969,2022-05-22 00:00:24+00:00,#NosUnimosONosJodemos\nLa revista Semana con s...,1528163832690003969,NaN,2022-05-22T00:00:00Z,25.0,2.0,58.0,0.0,None,#NosUnimosONosJodemos,None,None,42,2,0.047619
2,es,61589689,1528164772281196544,2022-05-22 00:04:08+00:00,#NosUnimosONosJodemos porque no queremos en Co...,1528164772281196544,NaN,2022-05-22T00:00:00Z,20.0,1.0,21.0,0.0,None,#NosUnimosONosJodemos,None,None,13,2,0.153846
3,es,80188601,1528164140308746240,2022-05-22 00:01:37+00:00,#NosUnimosONosJodemos Defender el país a toda ...,1528164140308746240,NaN,2022-05-22T00:00:00Z,2.0,0.0,2.0,0.0,None,#NosUnimosONosJodemos,None,None,12,2,0.166667
4,es,1388227835194707971,1528163741828882437,2022-05-22 00:00:02+00:00,2022-05-21 19:00\n#NosUnimosONosJodemos\n\na D...,1528163741828882437,NaN,2022-05-22T00:00:00Z,1.0,0.0,3.0,0.0,None,#NosUnimosONosJodemos,None,None,10,2,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290830,es,1275895677239857153,1539759865643601920,2022-06-22 23:58:53+00:00,#LosImpuestosDePetro deberían también ser apli...,1539759865643601920,NaN,2022-06-22T23:50:00Z,19.0,6.0,61.0,2.0,None,#LosImpuestosDePetro,None,None,48,1,0.020833
290831,es,756689409773277184,1539759241808027648,2022-06-22 23:56:25+00:00,TODO SE SUPO - SUCEDE A ESTA HORA - COLOMBIA H...,1539759241808027648,NaN,2022-06-22T23:50:00Z,0.0,0.0,1.0,0.0,None,#LosImpuestosDePetro,None,None,36,6,0.166667
290832,es,87009246,1539758385566031874,2022-06-22 23:53:01+00:00,Los veo muy preocupados por reforma tributaria...,1539758385566031874,NaN,2022-06-22T23:50:00Z,0.0,1.0,0.0,0.0,None,#LosImpuestosDePetro,None,None,43,1,0.023256
290833,es,760914847102070784,1539758097677312004,2022-06-22 23:51:52+00:00,#LosImpuestosDePetro para cuando los congresis...,1539758097677312004,NaN,2022-06-22T23:50:00Z,0.0,0.0,0.0,0.0,None,#LosImpuestosDePetro,None,None,21,1,0.047619


In [6]:
cleaned = pd.read_parquet('../Data/Full_dataset/data_filtered.gzip')

In [30]:
cleaned_political = cleaned.merge(trendings_keywords[['trending','Sector']].drop_duplicates(), how='left', left_on='Trending', right_on='trending')

In [31]:
cleaned_political['Sector'].value_counts()

Neutro       152594
Izquierda     86006
Derecha       52235
Name: Sector, dtype: int64